In [1]:
# Try to load dataset
from joblib import load

features_path = "C:/Users/lokix/Desktop/figshare_upload/psd_features_data_X"
labels_path = "C:/Users/lokix/Desktop/figshare_upload/labels_y"
master_path = "C:/Users/lokix/Desktop/figshare_upload/master_metadata_index.csv"

X = load(features_path, mmap_mode="r")
y = load(labels_path, mmap_mode="r")

In [2]:
import torch
import pandas as pd
import numpy as np
from EEGDataset import EEGDataset
from GNNModel import EEGGNN
from torchvision.transforms import Compose, ToTensor
from sklearn.model_selection import train_test_split
from torch_geometric.loader import DataLoader

In [3]:
SFREQ = 250.0
SEED = 42

MASTER_DATASET_INDEX = pd.read_csv(master_path)
subjects = MASTER_DATASET_INDEX["patient_ID"].astype("str").unique()
train_subjects, test_subjects = train_test_split(subjects, test_size=0.30, random_state=SEED)

C:\Users\lokix\AppData\Local\Temp\ipykernel_13032\1487538665.py:4: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  MASTER_DATASET_INDEX = pd.read_csv(master_path)


In [4]:
heldout_train_indices = MASTER_DATASET_INDEX.index[MASTER_DATASET_INDEX["patient_ID"].astype("str").isin(train_subjects)].tolist()
heldout_test_indices = MASTER_DATASET_INDEX.index[MASTER_DATASET_INDEX["patient_ID"].astype("str").isin(test_subjects)].tolist()

train_dataset = EEGDataset(X=X,
                     y=y,
                     indices=heldout_train_indices,
                     loader_type="heldout_test",
                     sfreq=SFREQ,
                     transform=Compose([ToTensor()]))

test_dataset = EEGDataset(X=X,
                     y=y,
                     indices=heldout_test_indices,
                     loader_type="heldout_test",
                     sfreq=SFREQ,
                     transform=Compose([ToTensor()]))


In [7]:
BATCH_SIZE = 8
NUM_WORKERS = 0
PIN_MEMORY = True

train_batches = DataLoader(dataset=train_dataset,
                           batch_size=BATCH_SIZE,
                           shuffle=False,
                           num_workers=NUM_WORKERS,
                           pin_memory=PIN_MEMORY)

test_batches = DataLoader(dataset=test_dataset,
                          batch_size=BATCH_SIZE,
                          shuffle=False,
                          num_workers=NUM_WORKERS,
                          pin_memory=PIN_MEMORY)


In [8]:
model = EEGGNN(True, SFREQ, BATCH_SIZE)
model = model.double()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.CrossEntropyLoss()


def train():
    model.train()

    for data in train_batches:  # Iterate in batches over the training dataset.
        data.batch = data.batch.view(data.batch.shape[0], -1)
        out = model(data.x, data.edge_index, data.edge_weight,
                    data.batch)  # Perform a single forward pass.
        loss = criterion(out, data.y)  # Compute the loss.
        loss.backward()  # Derive gradients.
        optimizer.step()  # Update parameters based on gradients.
        optimizer.zero_grad()  # Clear gradients.


def test(loader):
    model.eval()

    correct = 0
    for data in loader:  # Iterate in batches over the training/test dataset.
        out = model(data.x, data.edge_index, data.edge_weight, data.batch)
        pred = out.argmax(dim=1)  # Use the class with highest probability.
        correct += int(
            (pred == data.y).sum())  # Check against ground-truth labels.
    return correct / len(
        loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(2):
    train()
    train_acc = test(train_batches)
    test_acc = test(test_batches)
    print(
        f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}'
    )


IndexError: index 23809 is out of bounds for axis 0 with size 23794